In [120]:
sc

In [121]:
sc.version

'1.6.0'

In [63]:
sc.appName

'pyspark-shell'

In [64]:
sc.applicationId

'local-1491549814082'

In [65]:
sc.defaultParallelism

16

In [66]:
sc.defaultMinPartitions

2

In [67]:
sc.environment

{'PYTHONHASHSEED': '0'}

In [68]:
sc.master

'local[*]'

In [69]:
sc.startTime

1491549810921

In [70]:
sc.sparkUser()

'notebook'

In [71]:
sc.pythonVer

'3.4'

In [72]:
sc.PACKAGE_EXTENSIONS

('.zip', '.egg', '.jar')

In [73]:
sc.pythonExec

'python'

In [74]:
rdd = sc.parallelize(['I', 'will', 'complete', 'learning', 'Spark'])
rdd1 = sc.parallelize(['using', 'Jupyter' , 'Notebook', 'in', '20', 'days'],8)

In [75]:
rdd

ParallelCollectionRDD[58] at parallelize at PythonRDD.scala:423

In [76]:
rdd1

ParallelCollectionRDD[59] at parallelize at PythonRDD.scala:423

In [77]:
rdd.getNumPartitions()

16

In [78]:
rdd1.getNumPartitions()

8

In [79]:
rdd.toDebugString()

b'(16) ParallelCollectionRDD[58] at parallelize at PythonRDD.scala:423 []'

In [80]:
rdd1.toDebugString()

b'(8) ParallelCollectionRDD[59] at parallelize at PythonRDD.scala:423 []'

In [81]:
rdd2 = rdd1.coalesce(2)

In [82]:
rdd2.getNumPartitions()

2

# Lab 1 : MapR

In [83]:
# read auction data into an RDD
auctionRDD = sc.textFile("./data/MaprDataFiles/auctiondata.csv")

In [84]:
# explore data in the file by looking at first 10 lines using 'take' command 
auctionRDD.take(10)

['8213034705,95,2.927373,jake7870,0,95,117.5,xbox,3',
 '8213034705,115,2.943484,davidbresler2,1,95,117.5,xbox,3',
 '8213034705,100,2.951285,gladimacowgirl,58,95,117.5,xbox,3',
 '8213034705,117.5,2.998947,daysrus,10,95,117.5,xbox,3',
 '8213060420,2,0.065266,donnie4814,5,1,120,xbox,3',
 '8213060420,15.25,0.123218,myreeceyboy,52,1,120,xbox,3',
 '8213060420,3,0.186539,parakeet2004,5,1,120,xbox,3',
 '8213060420,10,0.18669,parakeet2004,5,1,120,xbox,3',
 '8213060420,24.99,0.187049,parakeet2004,5,1,120,xbox,3',
 '8213060420,20,0.249491,bluebubbles_1,25,1,120,xbox,3']

In [85]:
# if you see above output, we can observe that 
# 1. Output is a list of elements
# 2. each line in the file has become an element

# This is not usable for our analysis. We need to split each line into seperate fields based on comma 
# delimiter

auctionsRDD = auctionRDD.map(lambda line: line.split(","))

In [86]:
# lets take a look at data inside auctionsRDD
auctionsRDD.take(3)

[['8213034705', '95', '2.927373', 'jake7870', '0', '95', '117.5', 'xbox', '3'],
 ['8213034705',
  '115',
  '2.943484',
  'davidbresler2',
  '1',
  '95',
  '117.5',
  'xbox',
  '3'],
 ['8213034705',
  '100',
  '2.951285',
  'gladimacowgirl',
  '58',
  '95',
  '117.5',
  'xbox',
  '3']]

In [87]:
# If you see above output, it is a list of lists each list inside the main list represents a line in the file
# Each element inside the inner list represents a field or value

# Now, declare variables and assign positions so that we can refer various columns in the above RDD using names instead of numbers. 
# Ex: auctionsRDD[aucid] instead of auctionsRDD[0] 
# this approach is for best practice as it improves readablility.
aucid = 0
bid = 1
bidtime = 2
bidder = 3
bidderrate = 4
openbid = 5
price = 6
itemtype = 7
dtl = 8

#### Lets explore this dataset more !!

In [88]:
# 1. How do you see the first element of the auctionsRDD?
auctionsRDD.first()

['8213034705', '95', '2.927373', 'jake7870', '0', '95', '117.5', 'xbox', '3']

In [89]:
# 2. What do you use to see the first 5 elements of the RDD?
auctionsRDD.take(5)

[['8213034705', '95', '2.927373', 'jake7870', '0', '95', '117.5', 'xbox', '3'],
 ['8213034705',
  '115',
  '2.943484',
  'davidbresler2',
  '1',
  '95',
  '117.5',
  'xbox',
  '3'],
 ['8213034705',
  '100',
  '2.951285',
  'gladimacowgirl',
  '58',
  '95',
  '117.5',
  'xbox',
  '3'],
 ['8213034705',
  '117.5',
  '2.998947',
  'daysrus',
  '10',
  '95',
  '117.5',
  'xbox',
  '3'],
 ['8213060420', '2', '0.065266', 'donnie4814', '5', '1', '120', 'xbox', '3']]

In [90]:
# 3. What is the total number of bids?
totbids = auctionsRDD.count()
totbids

10654

In [91]:
# 4. What is the total number of distinct items that were auctioned?
totitems = auctionsRDD.map(lambda auction: auction[aucid]).distinct().count()
totitems

627

In [92]:
# 5. What is the total number of item types that were auctioned?
totitemtype = auctionsRDD.map(lambda auction:auction[itemtype]).distinct().count()
totitemtype

3

In [93]:
# 6. What is the total number of bids per item type?
bids_itemtype = auctionsRDD.map(lambda auction : (auction[itemtype], 1)).reduceByKey(lambda a,b:a+b)
bids_itemtype.collect()
#otherways
# totitemtype = auctionsRDD.map(lambda auction:(auction[itemtype],1)).countByKey()
# totitemtype = auctionsRDD.map(lambda auction:auction[itemtype]).countByValue()

[('xbox', 2784), ('palm', 5917), ('cartier', 1953)]

In [94]:
# 8. Across all auctioned items, what is the maximum number of bids?
maxbids = auctionsRDD.map(lambda auction: (auction[aucid],1)) \
                            .reduceByKey(lambda x,y:x+y) \
                            .map(lambda x: x[1]).reduce(max)
maxbids        

75

In [95]:
# 9. Across all auctioned items, what is the minimum number of bids?
minbids = auctionsRDD.map(lambda auction: (auction[aucid],1)) \
                            .reduceByKey(lambda x,y:x+y) \
                            .map(lambda x: x[1]).reduce(min)
minbids        

1

In [96]:
# 10. What is the average number of bids per auction? Hint: totalbids/totalauctions
#totalbids = auctionsRDD.map(lambda auction: (auction[aucid],1)).reduceByKey(lambda x,y:x+y).map(lambda x: x[1]).sum()
avgbids = totbids/totitems      
avgbids

16.992025518341308

# Caching RDDs:

In [97]:
#7. Create an RDD that contains total bids for each auction.
bidsAuctionRDD = auctionsRDD.map(lambda auction: (auction[aucid],1)).reduceByKey(lambda x,y:x+y)
bidsAuctionRDD.persist()    

PythonRDD[97] at RDD at PythonRDD.scala:43

In [98]:
bidsAuctionRDD.take(5)

[('3024579522', 7),
 ('3018964794', 20),
 ('3014314236', 5),
 ('8214279576', 4),
 ('1643136423', 10)]

In [99]:
bidsAuctionRDD.toDebugString()

b'(2) PythonRDD[97] at RDD at PythonRDD.scala:43 [Memory Serialized 1x Replicated]\n |       CachedPartitions: 1; MemorySize: 4.7 KB; ExternalBlockStoreSize: 0.0 B; DiskSize: 0.0 B\n |  MapPartitionsRDD[96] at mapPartitions at PythonRDD.scala:374 [Memory Serialized 1x Replicated]\n |  ShuffledRDD[95] at partitionBy at NativeMethodAccessorImpl.java:-2 [Memory Serialized 1x Replicated]\n +-(2) PairwiseRDD[94] at reduceByKey at <ipython-input-97-0d8d74e87bfd>:2 [Memory Serialized 1x Replicated]\n    |  PythonRDD[93] at reduceByKey at <ipython-input-97-0d8d74e87bfd>:2 [Memory Serialized 1x Replicated]\n    |  MapPartitionsRDD[62] at textFile at NativeMethodAccessorImpl.java:-2 [Memory Serialized 1x Replicated]\n    |  ./data/MaprDataFiles/auctiondata.csv HadoopRDD[61] at textFile at NativeMethodAccessorImpl.java:-2 [Memory Serialized 1x Replicated]'

In [100]:
maxbids = bidsAuctionRDD.map(lambda bid: bid[1]).reduce(max)
maxbids
# maxbids = bidsAuctionRDD.map(lambda bid: bid[1]).max()

75

In [101]:
minbids = bidsAuctionRDD.map(lambda bid: bid[1]).reduce(min)
minbids
# minbids = bidsAuctionRDD.map(lambda bid: bid[1]).min()

1

In [102]:
auctionsRDD.filter(lambda x: '1639672910' in x).collect()

[['1639672910',
  '5200',
  '6.291875',
  'esmodeus',
  '0',
  '5000',
  '5400',
  'cartier',
  '7'],
 ['1639672910',
  '5100',
  '6.925520833',
  'akapson',
  '7',
  '5000',
  '5400',
  'cartier',
  '7'],
 ['1639672910',
  '5300',
  '6.926469907',
  'akapson',
  '7',
  '5000',
  '5400',
  'cartier',
  '7'],
 ['1639672910',
  '5400',
  '6.933402778',
  'esmodeus',
  '0',
  '5000',
  '5400',
  'cartier',
  '7']]

In [103]:
maxbidamt = auctionsRDD.map(lambda x: (x[0],x[1])).max(key = lambda x : float(x[1]))
maxbidamt

('1639672910', '5400')

In [104]:
# COntents of an RDD inside each partition

a = sc.parallelize(range(1, 100))
# Return an RDD created by coalescing all elements within each partition into a list.
a.glom().collect()

[[1, 2, 3, 4, 5, 6],
 [7, 8, 9, 10, 11, 12],
 [13, 14, 15, 16, 17, 18],
 [19, 20, 21, 22, 23, 24],
 [25, 26, 27, 28, 29, 30],
 [31, 32, 33, 34, 35, 36, 37],
 [38, 39, 40, 41, 42, 43],
 [44, 45, 46, 47, 48, 49],
 [50, 51, 52, 53, 54, 55],
 [56, 57, 58, 59, 60, 61],
 [62, 63, 64, 65, 66, 67, 68],
 [69, 70, 71, 72, 73, 74],
 [75, 76, 77, 78, 79, 80],
 [81, 82, 83, 84, 85, 86],
 [87, 88, 89, 90, 91, 92],
 [93, 94, 95, 96, 97, 98, 99]]

## Lab 2

In [105]:
sqlContext

In [106]:
type(sqlContext)

pyspark.sql.context.HiveContext

In [107]:
from pyspark.sql import Row

In [108]:
# Load a text file and inferring the Schema Using Reflection
inputRDD = sc.textFile("./data/MaprDataFiles/auctiondata.csv").map(lambda lines: lines.split(","))
inputRDD.take(5)

[['8213034705', '95', '2.927373', 'jake7870', '0', '95', '117.5', 'xbox', '3'],
 ['8213034705',
  '115',
  '2.943484',
  'davidbresler2',
  '1',
  '95',
  '117.5',
  'xbox',
  '3'],
 ['8213034705',
  '100',
  '2.951285',
  'gladimacowgirl',
  '58',
  '95',
  '117.5',
  'xbox',
  '3'],
 ['8213034705',
  '117.5',
  '2.998947',
  'daysrus',
  '10',
  '95',
  '117.5',
  'xbox',
  '3'],
 ['8213060420', '2', '0.065266', 'donnie4814', '5', '1', '120', 'xbox', '3']]

In [109]:
# Parse each line and create a row with proper datatypes
auctionsRDD = inputRDD.map(lambda line: Row(aucid = int(line[0]), bid = float(line[1]), bidtime = float(line[2]), \
                                            bidder = line[3], bidderate = int(line[4]), openbid = float(line[5]), \
                                            price = float(line[6]), itemtype = line[7], dtl = int(line[8])))


In [110]:
type(inputRDD)

pyspark.rdd.PipelinedRDD

In [111]:
type(auctionsRDD)

pyspark.rdd.PipelinedRDD

In [112]:
auctionsRDD.toDebugString()

b'(2) PythonRDD[108] at RDD at PythonRDD.scala:43 []\n |  MapPartitionsRDD[106] at textFile at NativeMethodAccessorImpl.java:-2 []\n |  ./data/MaprDataFiles/auctiondata.csv HadoopRDD[105] at textFile at NativeMethodAccessorImpl.java:-2 []'

In [113]:
xboxRDD = auctionsRDD.map(lambda line : "xbox" in line[7])

In [114]:
xboxRDD.toDebugString()

b'(2) PythonRDD[109] at RDD at PythonRDD.scala:43 []\n |  MapPartitionsRDD[106] at textFile at NativeMethodAccessorImpl.java:-2 []\n |  ./data/MaprDataFiles/auctiondata.csv HadoopRDD[105] at textFile at NativeMethodAccessorImpl.java:-2 []'

In [115]:
# create a DataFrame for above RDD
auctionsDF = sqlContext.createDataFrame(auctionsRDD)

In [116]:
auctionsDF.show()

+----------+-----+--------------+---------+--------+---+--------+-------+-----+
|     aucid|  bid|        bidder|bidderate| bidtime|dtl|itemtype|openbid|price|
+----------+-----+--------------+---------+--------+---+--------+-------+-----+
|8213034705| 95.0|      jake7870|        0|2.927373|  3|    xbox|   95.0|117.5|
|8213034705|115.0| davidbresler2|        1|2.943484|  3|    xbox|   95.0|117.5|
|8213034705|100.0|gladimacowgirl|       58|2.951285|  3|    xbox|   95.0|117.5|
|8213034705|117.5|       daysrus|       10|2.998947|  3|    xbox|   95.0|117.5|
|8213060420|  2.0|    donnie4814|        5|0.065266|  3|    xbox|    1.0|120.0|
|8213060420|15.25|   myreeceyboy|       52|0.123218|  3|    xbox|    1.0|120.0|
|8213060420|  3.0|  parakeet2004|        5|0.186539|  3|    xbox|    1.0|120.0|
|8213060420| 10.0|  parakeet2004|        5| 0.18669|  3|    xbox|    1.0|120.0|
|8213060420|24.99|  parakeet2004|        5|0.187049|  3|    xbox|    1.0|120.0|
|8213060420| 20.0| bluebubbles_1|       

In [117]:
auctionsDF.printSchema()

root
 |-- aucid: long (nullable = true)
 |-- bid: double (nullable = true)
 |-- bidder: string (nullable = true)
 |-- bidderate: long (nullable = true)
 |-- bidtime: double (nullable = true)
 |-- dtl: long (nullable = true)
 |-- itemtype: string (nullable = true)
 |-- openbid: double (nullable = true)
 |-- price: double (nullable = true)



In [118]:
# Register dataframe as a table
auctionsDF.registerTempTable("auctionTbl")

In [119]:
type(auctionTbl)

NameError: name 'auctionTbl' is not defined

In [ ]:
sqlContext.sql("show tables").show()

In [ ]:
auctionsDF.count()

In [ ]:
auctionsDF.cache()

In [ ]:
auctionsDF.describe()

In [ ]:
auctionsDF.columns

In [ ]:
auctionsDF.groupby("itemtype").count().show()

In [ ]:
auctionsDF.select("itemtype").distinct().show()

## Lets answer few questions about data in this dataframe

In [ ]:
#1. What is the total number of bids?
auctionsDF.count()

In [ ]:
#2. What is the number of distinct auctions?
auctionsDF.select("aucid").distinct().count()

In [ ]:
#3. What is the number of distinct itemtypes?
auctionsDF.select("itemtype").distinct().count()

In [ ]:
#4. We would like a count of bids per auction and the item type (as shown below). How would you do this?
auctionsDF.groupBy("itemtype", "aucid").count().show()

In [ ]:
# 5. For each auction item and item type, we want the max, min and average number of bids.
import pyspark.sql.functions as func
auctionsDF.groupBy("itemtype","aucid").count().agg(func.max("count").alias("Max"), func.min("count").alias("Min"), func.avg("count").alias("Count")).show()

In [ ]:
# 6. For each auction item and item type, we want the max, min and average of a bid:
auctionsDF.groupBy("itemtype","aucid").agg(func.max("bid").alias("MaximumBid"), func.min("bid").alias("MinimumBid"), func.avg("bid").alias("AvgBid")).show()

In [ ]:
# 7. What is the number of auctions with final price greater than 200?
auctionsDF.filter(auctionsDF.price > 200).count()

In [ ]:
xboxes = sqlContext.sql("select * from auctiontbl where itemtype = 'xbox'")
xboxes.show()

In [ ]:
from pyspark.sql.functions import col
auctionsDF.select(auctionsDF.aucid.alias("Auction ID"),auctionsDF.bid ).filter(col('bid') > 100).show()

In [ ]:
auctionsDF.crosstab("itemtype", "dtl").show()

# Others

In [ ]:
# Multiple Aggregate operations on multiple columns of a spark dataframe

from pyspark.sql.functions import mean, sum, max, col

groupBy = ["itemtype"]
aggregate = ["bid", "price"] 
funs = [mean, sum, max]

exprs = [f(c) for f in funs for c in aggregate]

# or equivalent df.groupby(groupBy).agg(*exprs)
auctionsDF.groupby(*groupBy).agg(*exprs).show()

In [ ]:
# 1. Random Data Generation
import pyspark.sql.functions as f
df = sqlContext.range(0,10)

In [ ]:
type(df)

In [ ]:
# Generate two other columns using uniform distribution and normal distribution.
df.select("id", f.rand(seed=10).alias("uniform"), f.randn(seed=27).alias("normal")).show()

In [ ]:
# A slightly different way to generate the two random columns
df = sqlContext.range(0, 10) \
               .withColumn('uniform', f.rand(seed=10)) \
               .withColumn('normal', f.randn(seed=27))

In [ ]:
#2. Summary and Descriptive Statistics
df.describe().show()

In [ ]:
# describe on a subset of the columns
df.describe('uniform', 'normal').show()

In [ ]:
from pyspark.sql.functions import mean, min, max
df.select(mean('uniform'), min('uniform'), max('uniform')).show()

In [ ]:
# 3. Sample covariance and correlation.
"""
Covariance is a measure of how two variables change with respect to each other. 
A positive number would mean that there is a tendency that as one variable increases, the other increases as well. 
A negative number would mean that as one variable increases, the other variable has a tendency to decrease. 
The sample covariance of two columns of a DataFrame can be calculated as follows:
"""
df.stat.cov('uniform', 'normal')

In [ ]:
df.stat.cov('id', 'id')

In [ ]:
# Correlation is a normalized measure of covariance that is easier to understand,
# as it provides quantitative measurements of the statistical dependence between two random variables.
df.stat.corr('uniform', 'normal')

In [ ]:
df.stat.corr('id', 'id')

In [ ]:
# Create a DataFrame with two columns (name, item) 
names = ["Alice", "Bob", "Mike"]
items = ["milk", "bread", "butter", "apples", "oranges"]
df = sqlContext.createDataFrame([(names[i % 3], items[i % 5]) for i in range(100)], ["name", "item"])

In [ ]:
# Take a look at the first 10 rows.
df.show(10)

In [ ]:
df.stat.crosstab("name", "item").show()

In [ ]:
freq = df.stat.freqItems(["name","item"],0.5)

In [ ]:
freq.show()

# Supplimentary Mapr

In [ ]:
from pyspark.sql import Row
sfpdRDD = sc.textFile("./data/MaprDataFiles/sfpd.csv") \
        .map(lambda line:line.split(",")) \
        .map(lambda i: Row(IncidentNum = int(i[0]), Category = i[1], Descript = i[2], DayOfWeek = i[3] \
                         , Date = i[4], Time = i[5], PdDistrict = i[6], Resolution = i[7] \
                         , Address = i[8], XCoord = i[9], YCoord = i[10], PdID = int(i[11])))
sfpdRDD.take(5)

In [ ]:
sfpdDF = sfpdRDD.toDF()

In [ ]:
sfpdDF.printSchema()

In [ ]:
sfpdDF.registerTempTable("sfpdtbl")

### Seeing the first few lines of the DataFrame

In [ ]:
sfpdDF.head(5)

In [ ]:
sfpdDF.show(5)

In [ ]:
sfpdDF.take(5)

In [ ]:
sqlContext.sql("select * from sfpdtbl limit 5").show()

In [ ]:
# Print the schema in a tree format
sfpdDF.printSchema()

In [ ]:
# 1. What is the total number of incidents?
sfpdRDD.count()

In [ ]:
sfpdDF.count()

In [ ]:
# 2. How many categories are there?
sfpdRDD.map(lambda row: row[1]).distinct().count()

In [ ]:
sfpdDF.select("Category").distinct().count()

In [ ]:
#3. How many Pd Districts are there?
sfpdRDD.map(lambda x:x[6]).distinct().count()

In [ ]:
sfpdDF.select("PdDistrict").distinct().count()

In [ ]:
# What are the different districts?
sfpdRDD.map(lambda x: x[6]).distinct().collect()

In [ ]:
sfpdDF.select("PdDistrict").distinct().show()

In [ ]:
# How many incidents were there in the Tenderloin district?
sfpdRDD.filter(lambda x: "TENDERLOIN" in (x[6]).upper()).count()

In [ ]:
import pyspark.sql.functions as f
sfpdDF.filter(f.upper(sfpdDF.PdDistrict) == 'TENDERLOIN').count()

In [ ]:
# How many incidents were there in the Richmond district?
sfpdDF.filter(f.lower(sfpdDF.PdDistrict) == 'richmond').count()

In [ ]:
# Get all the categories on incidents
sfpdDF.select("Category").distinct().show()

In [ ]:
sfpdDF.select(sfpdDF.Category.alias("CAT")).distinct().show()

In [ ]:
sqlContext.sql("""
               SELECT DISTINCT Category 
                 FROM sfpdtbl
               """).show()

In [ ]:
sfpdDF.select(sfpdDF["Category"].alias("CAT")).distinct().show()

In [ ]:
# Get all the districts
from pyspark.sql.functions import col
sfpdDF.select(col("PdDistrict").alias("District")).distinct().show()

# SFPD Json

In [131]:
sfpdJDF = sqlContext.read.json("/resources/data/MaprDataFiles/sfpd.json")
type(sfpdJDF)

pyspark.sql.dataframe.DataFrame

In [132]:
sfpdJDF.printSchema()

root
 |-- Address: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- Descript: string (nullable = true)
 |-- IncidentNum: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- PdDistrict: string (nullable = true)
 |-- PdId: long (nullable = true)
 |-- Resolution: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)



In [133]:
sfpdJDF.columns

['Address',
 'Category',
 'Date',
 'DayOfWeek',
 'Descript',
 'IncidentNum',
 'Location',
 'PdDistrict',
 'PdId',
 'Resolution',
 'Time',
 'X',
 'Y']

In [134]:
sfpdJDF.persist()

DataFrame[Address: string, Category: string, Date: string, DayOfWeek: string, Descript: string, IncidentNum: string, Location: string, PdDistrict: string, PdId: bigint, Resolution: string, Time: string, X: double, Y: double]

In [144]:
# Get incidents and category and sort by category in desc order
#sfpdJDF.select("Category", "IncidentNum").orderBy(sfpdJDF['Category'].desc()).show()
#sfpdJDF.select("Category", "IncidentNum").sort(sfpdJDF['Category'].desc()).show()
sfpdJDF.select("Category", "IncidentNum").orderBy(sfpdJDF["Category"], ascending = False).show()

+-----------+-----------+
|   Category|IncidentNum|
+-----------+-----------+
|WEAPON LAWS|  110058129|
|WEAPON LAWS|  110035149|
|WEAPON LAWS|  110058129|
|WEAPON LAWS|  110062697|
|WEAPON LAWS|  110054478|
|WEAPON LAWS|  110060403|
|WEAPON LAWS|  110054644|
|WEAPON LAWS|  110049849|
|WEAPON LAWS|  110048835|
|WEAPON LAWS|  110061495|
|WEAPON LAWS|  110043249|
|WEAPON LAWS|  110059597|
|WEAPON LAWS|  110041033|
|WEAPON LAWS|  110053555|
|WEAPON LAWS|  110038331|
|WEAPON LAWS|  110052438|
|WEAPON LAWS|  110036460|
|WEAPON LAWS|  110050711|
|WEAPON LAWS|  110036460|
|WEAPON LAWS|  110054359|
+-----------+-----------+
only showing top 20 rows



In [137]:
sfpdJDF.registerTempTable("sfpdtbl")

In [138]:
sqlContext.sql("SELECT Category, IncidentNum from sfpdtbl order by Category desc").show()

+-----------+-----------+
|   Category|IncidentNum|
+-----------+-----------+
|WEAPON LAWS|  110058129|
|WEAPON LAWS|  110035149|
|WEAPON LAWS|  110058129|
|WEAPON LAWS|  110062697|
|WEAPON LAWS|  110054478|
|WEAPON LAWS|  110060403|
|WEAPON LAWS|  110054644|
|WEAPON LAWS|  110049849|
|WEAPON LAWS|  110048835|
|WEAPON LAWS|  110061495|
|WEAPON LAWS|  110043249|
|WEAPON LAWS|  110059597|
|WEAPON LAWS|  110041033|
|WEAPON LAWS|  110053555|
|WEAPON LAWS|  110038331|
|WEAPON LAWS|  110052438|
|WEAPON LAWS|  110036460|
|WEAPON LAWS|  110050711|
|WEAPON LAWS|  110036460|
|WEAPON LAWS|  110054359|
+-----------+-----------+
only showing top 20 rows



In [146]:
from pyspark.sql.functions import col
sfpdJDF.select("Category", "IncidentNum").sort(col("Category").desc()).show()

+-----------+-----------+
|   Category|IncidentNum|
+-----------+-----------+
|WEAPON LAWS|  110058129|
|WEAPON LAWS|  110035149|
|WEAPON LAWS|  110058129|
|WEAPON LAWS|  110062697|
|WEAPON LAWS|  110054478|
|WEAPON LAWS|  110060403|
|WEAPON LAWS|  110054644|
|WEAPON LAWS|  110049849|
|WEAPON LAWS|  110048835|
|WEAPON LAWS|  110061495|
|WEAPON LAWS|  110043249|
|WEAPON LAWS|  110059597|
|WEAPON LAWS|  110041033|
|WEAPON LAWS|  110053555|
|WEAPON LAWS|  110038331|
|WEAPON LAWS|  110052438|
|WEAPON LAWS|  110036460|
|WEAPON LAWS|  110050711|
|WEAPON LAWS|  110036460|
|WEAPON LAWS|  110054359|
+-----------+-----------+
only showing top 20 rows



In [153]:
# Get number of incidents by category and sort by count desc
from pyspark.sql.functions import col
sfpdJDF.groupBy("Category").count().sort(col("count").desc()).show()

+--------------------+------+
|            Category| count|
+--------------------+------+
|       LARCENY/THEFT|150511|
|      OTHER OFFENSES| 88176|
|        NON-CRIMINAL| 81882|
|             ASSAULT| 55700|
|           VANDALISM| 32656|
|            WARRANTS| 29914|
|       DRUG/NARCOTIC| 27999|
|       VEHICLE THEFT| 27842|
|            BURGLARY| 26339|
|      SUSPICIOUS OCC| 25656|
|      MISSING PERSON| 20582|
|             ROBBERY| 16854|
|               FRAUD| 12662|
|     SECONDARY CODES|  8587|
|         WEAPON LAWS|  6713|
|            TRESPASS|  5191|
|     STOLEN PROPERTY|  4476|
|FORGERY/COUNTERFE...|  3829|
|SEX OFFENSES, FOR...|  3513|
|        PROSTITUTION|  3269|
+--------------------+------+
only showing top 20 rows



In [158]:
sfpdJDF.groupBy("Category").count().orderBy("count", ascending = False).show()

+--------------------+------+
|            Category| count|
+--------------------+------+
|       LARCENY/THEFT|150511|
|      OTHER OFFENSES| 88176|
|        NON-CRIMINAL| 81882|
|             ASSAULT| 55700|
|           VANDALISM| 32656|
|            WARRANTS| 29914|
|       DRUG/NARCOTIC| 27999|
|       VEHICLE THEFT| 27842|
|            BURGLARY| 26339|
|      SUSPICIOUS OCC| 25656|
|      MISSING PERSON| 20582|
|             ROBBERY| 16854|
|               FRAUD| 12662|
|     SECONDARY CODES|  8587|
|         WEAPON LAWS|  6713|
|            TRESPASS|  5191|
|     STOLEN PROPERTY|  4476|
|FORGERY/COUNTERFE...|  3829|
|SEX OFFENSES, FOR...|  3513|
|        PROSTITUTION|  3269|
+--------------------+------+
only showing top 20 rows



In [160]:
import pyspark.sql.functions as f
sfpdJDF.groupBy("Category").agg(f.count("IncidentNum").alias("IncidentCount"))\
    .orderBy("`IncidentCount`", ascending = False).show()

+--------------------+-------------+
|            Category|IncidentCount|
+--------------------+-------------+
|       LARCENY/THEFT|       150511|
|      OTHER OFFENSES|        88176|
|        NON-CRIMINAL|        81882|
|             ASSAULT|        55700|
|           VANDALISM|        32656|
|            WARRANTS|        29914|
|       DRUG/NARCOTIC|        27999|
|       VEHICLE THEFT|        27842|
|            BURGLARY|        26339|
|      SUSPICIOUS OCC|        25656|
|      MISSING PERSON|        20582|
|             ROBBERY|        16854|
|               FRAUD|        12662|
|     SECONDARY CODES|         8587|
|         WEAPON LAWS|         6713|
|            TRESPASS|         5191|
|     STOLEN PROPERTY|         4476|
|FORGERY/COUNTERFE...|         3829|
|SEX OFFENSES, FOR...|         3513|
|        PROSTITUTION|         3269|
+--------------------+-------------+
only showing top 20 rows



In [162]:
sfpdJDF.groupBy("Category").agg(f.count("IncidentNum").alias("IncidentCount"))\
    .sort(col("IncidentCount").desc()).show()

+--------------------+-------------+
|            Category|IncidentCount|
+--------------------+-------------+
|       LARCENY/THEFT|       150511|
|      OTHER OFFENSES|        88176|
|        NON-CRIMINAL|        81882|
|             ASSAULT|        55700|
|           VANDALISM|        32656|
|            WARRANTS|        29914|
|       DRUG/NARCOTIC|        27999|
|       VEHICLE THEFT|        27842|
|            BURGLARY|        26339|
|      SUSPICIOUS OCC|        25656|
|      MISSING PERSON|        20582|
|             ROBBERY|        16854|
|               FRAUD|        12662|
|     SECONDARY CODES|         8587|
|         WEAPON LAWS|         6713|
|            TRESPASS|         5191|
|     STOLEN PROPERTY|         4476|
|FORGERY/COUNTERFE...|         3829|
|SEX OFFENSES, FOR...|         3513|
|        PROSTITUTION|         3269|
+--------------------+-------------+
only showing top 20 rows



In [164]:
# Count of incidents by Category and District
sfpdJDF.groupBy("Category","PdDistrict").count().show()

+--------------------+----------+-----+
|            Category|PdDistrict|count|
+--------------------+----------+-----+
|      SUSPICIOUS OCC|   MISSION| 3159|
|                TREA|  SOUTHERN|    1|
|  DISORDERLY CONDUCT|   TARAVAL|   80|
|SEX OFFENSES, FOR...|   MISSION|  602|
|        PROSTITUTION|   MISSION| 1763|
|         WEAPON LAWS|TENDERLOIN|  603|
|     SECONDARY CODES|   CENTRAL|  580|
|            GAMBLING|   MISSION|   14|
|   RECOVERED VEHICLE|      PARK|   97|
|       LARCENY/THEFT|  SOUTHERN|37497|
|             ASSAULT|   BAYVIEW| 7194|
|             ROBBERY|   TARAVAL|  941|
|         DRUNKENNESS|      PARK|  209|
|   RECOVERED VEHICLE|  NORTHERN|  209|
|PORNOGRAPHY/OBSCE...|      PARK|    1|
|         DRUNKENNESS|  NORTHERN|  198|
|       VEHICLE THEFT|      PARK| 2050|
|DRIVING UNDER THE...| INGLESIDE|  178|
|                TREA|   BAYVIEW|    2|
|     STOLEN PROPERTY|  RICHMOND|  202|
+--------------------+----------+-----+
only showing top 20 rows



In [165]:
# Counts of all the incidents by category and district
sfpdJDF.crosstab("Category", "PdDistrict").show()

+--------------------+-------+--------+-------+----------+-------+----+-------+--------+---------+--------+
| Category_PdDistrict|MISSION|SOUTHERN|TARAVAL|TENDERLOIN|CENTRAL|PARK|BAYVIEW|NORTHERN|INGLESIDE|RICHMOND|
+--------------------+-------+--------+-------+----------+-------+----+-------+--------+---------+--------+
|           VANDALISM|   3963|    5379|   3252|      1252|   3594|1976|   3729|    3594|     3667|    2250|
|                TREA|      1|       1|      2|         0|      2|   0|      2|       1|        0|       0|
|     FAMILY OFFENSES|    129|      32|     24|        20|     19|   8|     33|      12|       38|      15|
|          BAD CHECKS|     18|      33|     24|         4|     26|   7|     12|      20|       10|      15|
|PORNOGRAPHY/OBSCE...|      0|       4|      1|         1|      1|   1|      3|       0|        2|       1|
|      OTHER OFFENSES|  12495|   14757|   6713|      7840|   7020|4556|  11981|    8446|     9772|    4596|
|             BRIBERY|     6

In [166]:
# How many resolutions by type for each district?
sfpdJDF.groupBy(sfpdJDF.PdDistrict.alias("District"),sfpdJDF.Resolution.alias("Resolutions")) \
        .count() \
        .orderBy("District",ascending = True) \
        .orderBy("Resolutions", ascending = True) \
        .show()

+----------+--------------+-----+
|  District|   Resolutions|count|
+----------+--------------+-----+
|  SOUTHERN|ARREST, BOOKED|26755|
|   BAYVIEW|ARREST, BOOKED|14766|
|   MISSION|ARREST, BOOKED|23230|
|  NORTHERN|ARREST, BOOKED|14282|
|TENDERLOIN|ARREST, BOOKED|23521|
|  RICHMOND|ARREST, BOOKED| 5116|
|      PARK|ARREST, BOOKED| 7552|
|   CENTRAL|ARREST, BOOKED|11000|
| INGLESIDE|ARREST, BOOKED|10079|
|   TARAVAL|ARREST, BOOKED| 6987|
|   BAYVIEW| ARREST, CITED| 6230|
|   TARAVAL| ARREST, CITED| 2556|
| INGLESIDE| ARREST, CITED| 4509|
|  SOUTHERN| ARREST, CITED| 9129|
|   CENTRAL| ARREST, CITED| 3802|
|   MISSION| ARREST, CITED| 7461|
|      PARK| ARREST, CITED| 2749|
|  NORTHERN| ARREST, CITED| 4219|
|TENDERLOIN| ARREST, CITED| 4831|
|  RICHMOND| ARREST, CITED| 1676|
+----------+--------------+-----+
only showing top 20 rows



In [175]:
sfpdJDF.groupBy(sfpdJDF.PdDistrict.alias("District"),sfpdJDF.Resolution.alias("Resolutions")) \
        .count() \
        .orderBy(col("District").asc(),col("Resolutions").desc()) \
        .show()

+--------+--------------------+-----+
|District|         Resolutions|count|
+--------+--------------------+-----+
| BAYVIEW|           UNFOUNDED|  860|
| BAYVIEW|   PSYCHOPATHIC CASE|  840|
| BAYVIEW|PROSECUTED FOR LE...|    8|
| BAYVIEW|PROSECUTED BY OUT...|  108|
| BAYVIEW|      NOT PROSECUTED|  147|
| BAYVIEW|                NONE|35806|
| BAYVIEW|             LOCATED| 2594|
| BAYVIEW|   JUVENILE DIVERTED|   58|
| BAYVIEW|      JUVENILE CITED|  276|
| BAYVIEW|     JUVENILE BOOKED|  748|
| BAYVIEW| JUVENILE ADMONISHED|   97|
| BAYVIEW|EXCEPTIONAL CLEAR...|  167|
| BAYVIEW|DISTRICT ATTORNEY...|  370|
| BAYVIEW|COMPLAINANT REFUS...|  955|
| BAYVIEW|CLEARED-CONTACT J...|   44|
| BAYVIEW|       ARREST, CITED| 6230|
| BAYVIEW|      ARREST, BOOKED|14766|
| CENTRAL|           UNFOUNDED|  755|
| CENTRAL|   PSYCHOPATHIC CASE| 1335|
| CENTRAL|PROSECUTED FOR LE...|    1|
+--------+--------------------+-----+
only showing top 20 rows



In [176]:
from pyspark.sql.functions import col

(sfpdJDF.groupBy(sfpdJDF.PdDistrict.alias("District"),sfpdJDF.Resolution.alias("Resolutions")) 
    .count() 
    .filter("`count` >= 40000")
    .sort(col("count").desc()).show())

+--------+-----------+-----+
|District|Resolutions|count|
+--------+-----------+-----+
|SOUTHERN|       NONE|76721|
|NORTHERN|       NONE|53272|
| MISSION|       NONE|48786|
| CENTRAL|       NONE|48468|
+--------+-----------+-----+



In [177]:
from pyspark.sql.functions import desc

(sfpdJDF.groupBy(sfpdJDF.PdDistrict.alias("District"),sfpdJDF.Resolution.alias("Resolutions")) 
    .count()
    .filter("`count` >= 40000")
    .sort(desc("count")).show())

+--------+-----------+-----+
|District|Resolutions|count|
+--------+-----------+-----+
|SOUTHERN|       NONE|76721|
|NORTHERN|       NONE|53272|
| MISSION|       NONE|48786|
| CENTRAL|       NONE|48468|
+--------+-----------+-----+



In [179]:
sfpdJDF.crosstab("Category", "DayOfWeek").show()

+--------------------+--------+---------+------+------+------+--------+-------+
|  Category_DayOfWeek|Thursday|Wednesday|Friday|Sunday|Monday|Saturday|Tuesday|
+--------------------+--------+---------+------+------+------+--------+-------+
|           VANDALISM|    4285|     4322|  5127|  4947|  4338|    5343|   4294|
|                TREA|       0|        1|     1|     0|     4|       2|      1|
|     FAMILY OFFENSES|      41|       50|    66|    31|    45|      41|     56|
|          BAD CHECKS|      35|       35|    28|     9|    32|      11|     19|
|PORNOGRAPHY/OBSCE...|       1|        1|     2|     2|     3|       1|      4|
|      OTHER OFFENSES|   12642|    13496| 13489| 10993| 12334|   12413|  12809|
|             BRIBERY|      35|       39|    46|    34|    35|      42|     34|
|            TRESPASS|     716|      819|   689|   649|   817|     649|    852|
|           LOITERING|      49|       58|    28|    26|    82|      34|     95|
|DRIVING UNDER THE...|     224|      203

In [180]:
sfpdJDF.crosstab("PdDistrict","DayOfWeek").show()

+--------------------+---------+------+------+--------+-------+------+--------+
|PdDistrict_DayOfWeek|Wednesday|Monday|Friday|Saturday|Tuesday|Sunday|Thursday|
+--------------------+---------+------+------+--------+-------+------+--------+
|            SOUTHERN|    17750| 16156| 19336|   18574|  16923| 16336|   17457|
|           INGLESIDE|     8453|  8247|  8595|    7982|   8220|  7445|    8194|
|          TENDERLOIN|     8554|  7645|  7585|    7351|   8142|  6816|    8144|
|             MISSION|    12408| 11883| 13161|   12787|  12040| 12070|   12261|
|             TARAVAL|     7032|  6825|  7449|    6440|   6815|  6106|    6511|
|            RICHMOND|     5018|  4887|  5433|    5045|   4921|  4783|    4989|
|            NORTHERN|    10942| 10130| 11951|   11874|  10446| 10583|   10717|
|                PARK|     5820|  5528|  5955|    5463|   5353|  5402|    5643|
|             CENTRAL|     9261|  8780| 10669|   11196|   8659|  9713|    9072|
|             BAYVIEW|     9445|  9036| 

In [182]:
(sfpdJDF.filter((sfpdJDF.PdDistrict == "SOUTHERN") & (sfpdJDF.DayOfWeek == "Saturday"))
        .cube("PdDistrict", "DayOfWeek", "Category")
        .count()
        .withColumnRenamed("count", "IncCnt") 
        .filter("IncCnt > 1000")
        .dropna()
        .sort(col("IncCnt").desc())
        .withColumnRenamed("PdDistrict", "District")
        .withColumnRenamed("DayOfWeek", "Day Of Week")
        .show())

+--------+-----------+--------------+------+
|District|Day Of Week|      Category|IncCnt|
+--------+-----------+--------------+------+
|SOUTHERN|   Saturday| LARCENY/THEFT|  6197|
|SOUTHERN|   Saturday|  NON-CRIMINAL|  2383|
|SOUTHERN|   Saturday|OTHER OFFENSES|  2227|
|SOUTHERN|   Saturday|       ASSAULT|  1443|
+--------+-----------+--------------+------+



In [183]:
#What are the top 10 Resolutions ?
sqlContext.sql("SELECT resolution, count(1) as rescount from sfpdtbl group by resolution order by rescount desc limit 10").show()

+--------------------+--------+
|          resolution|rescount|
+--------------------+--------+
|                NONE|  403414|
|      ARREST, BOOKED|  143288|
|       ARREST, CITED|   47162|
|   PSYCHOPATHIC CASE|   15141|
|             LOCATED|   14628|
|           UNFOUNDED|    7055|
|COMPLAINANT REFUS...|    5142|
|     JUVENILE BOOKED|    4427|
|      JUVENILE CITED|    2593|
|EXCEPTIONAL CLEAR...|    1892|
+--------------------+--------+



In [184]:
(sfpdJDF
 .groupBy("Resolution")
 .count()
 .withColumnRenamed("count", "ResolutionCount")
 .withColumnRenamed("Resolution", "ResolutionType")
 .orderBy("ResolutionCount", ascending = False)
 .show(10)
)

+--------------------+---------------+
|      ResolutionType|ResolutionCount|
+--------------------+---------------+
|                NONE|         403414|
|      ARREST, BOOKED|         143288|
|       ARREST, CITED|          47162|
|   PSYCHOPATHIC CASE|          15141|
|             LOCATED|          14628|
|           UNFOUNDED|           7055|
|COMPLAINANT REFUS...|           5142|
|     JUVENILE BOOKED|           4427|
|      JUVENILE CITED|           2593|
|EXCEPTIONAL CLEAR...|           1892|
+--------------------+---------------+
only showing top 10 rows



In [185]:
# What are the top 10 most incident Categories?
sqlContext.sql("SELECT category, count(1) as inccount from sfpdtbl group by category order by inccount desc limit 10").show()

+--------------+--------+
|      category|inccount|
+--------------+--------+
| LARCENY/THEFT|  150511|
|OTHER OFFENSES|   88176|
|  NON-CRIMINAL|   81882|
|       ASSAULT|   55700|
|     VANDALISM|   32656|
|      WARRANTS|   29914|
| DRUG/NARCOTIC|   27999|
| VEHICLE THEFT|   27842|
|      BURGLARY|   26339|
|SUSPICIOUS OCC|   25656|
+--------------+--------+



In [191]:
sfpdJDF.groupBy(sfpdJDF["Category"]).count().sort(col("count").desc()).show(10)

+--------------+------+
|      Category| count|
+--------------+------+
| LARCENY/THEFT|150511|
|OTHER OFFENSES| 88176|
|  NON-CRIMINAL| 81882|
|       ASSAULT| 55700|
|     VANDALISM| 32656|
|      WARRANTS| 29914|
| DRUG/NARCOTIC| 27999|
| VEHICLE THEFT| 27842|
|      BURGLARY| 26339|
|SUSPICIOUS OCC| 25656|
+--------------+------+
only showing top 10 rows



In [192]:
sfpdJDF.explain()

== Physical Plan ==
InMemoryColumnarTableScan [Address#31,Category#32,Date#33,DayOfWeek#34,Descript#35,IncidentNum#36,Location#37,PdDistrict#38,PdId#39L,Resolution#40,Time#41,X#42,Y#43], InMemoryRelation [Address#31,Category#32,Date#33,DayOfWeek#34,Descript#35,IncidentNum#36,Location#37,PdDistrict#38,PdId#39L,Resolution#40,Time#41,X#42,Y#43], true, 10000, StorageLevel(false, true, false, false, 1), Scan JSONRelation[Address#31,Category#32,Date#33,DayOfWeek#34,Descript#35,IncidentNum#36,Location#37,PdDistrict#38,PdId#39L,Resolution#40,Time#41,X#42,Y#43] InputPaths: file:/resources/data/MaprDataFiles/sfpd.json, None


In [208]:
sfpdJDF.selectExpr("IncidentNum * 4 as INC").show(4)

+------------+
|         INC|
+------------+
|6.01784908E8|
|6.01784908E8|
|  6.017782E8|
|  6.017782E8|
+------------+
only showing top 4 rows



In [210]:
sfpdJDF.dtypes

[('Address', 'string'),
 ('Category', 'string'),
 ('Date', 'string'),
 ('DayOfWeek', 'string'),
 ('Descript', 'string'),
 ('IncidentNum', 'string'),
 ('Location', 'string'),
 ('PdDistrict', 'string'),
 ('PdId', 'bigint'),
 ('Resolution', 'string'),
 ('Time', 'string'),
 ('X', 'double'),
 ('Y', 'double')]

# Supp Lab

In [ ]:
#1. We define the mapping for our input variables. While this isn’t a necessary step, it makes it easier to refer to the different fields by names.
IncidntNum = 0
Category = 1
Descript = 2
DayOfWeek = 3
Date = 4
Time = 5
PdDistrict = 6
Resolution = 7
Address = 8
X = 9
Y = 10
PdId = 11

In [ ]:
sfpdRDD = sc.textFile("/resources/data/MaprDataFiles/sfpd.csv").map(lambda x: x.split(","))

In [ ]:
sfpdRDD.first()

In [ ]:
sfpdRDD.take(5)

In [ ]:
totincs = sfpdRDD.count()
totincs

In [ ]:
totres = sfpdRDD.map(lambda inc: inc[Resolution]).distinct().count()
totres

In [ ]:
PdDistricts = sfpdRDD.map(lambda x: (x[PdDistrict],1)).reduceByKey(lambda v1,v2:v1+v2).map(lambda x: (x[1],x[0])).sortByKey(False)
#a =PdDistricts.sortByKey(False, keyfunc=lambda x: x[1])
#a.collect()
PdDistricts.collect()

In [ ]:
PdDistricts = sfpdRDD.map(lambda x: (x[PdDistrict],1)).reduceByKey(lambda v1,v2:v1+v2)
#a =PdDistricts.sortByKey(False, keyfunc=lambda x: x[1])
#a.collect()
PdDistricts.sortBy(lambda x: x[1], False).collect()

In [ ]:
PdDistricts = sfpdRDD.map(lambda x: (x[PdDistrict],1)).reduceByKey(lambda v1,v2:v1+v2)
#a =PdDistricts.sortByKey(False, keyfunc=lambda x: x[1])
#a.collect()
PdDistricts.sortByKey(False, lambda x: x[1]).collect()

# Joins

In [1]:
addCatRdd = (sc.textFile("/resources/data/MaprDataFiles/J_AddCat.csv")
                .map(lambda line : line.split(","))
                .map(lambda r: (r[1],r[0])))
addCatRdd.take(5)

[('100_Block_of_JEFFERSON_ST', 'EMBEZZLEMENT'),
 ('SUTTER_ST/MASON_ST', 'EMBEZZLEMENT'),
 ('0_Block_of_SOUTHPARK_AV', 'BRIBERY'),
 ('1500_Block_of_15TH_ST', 'EMBEZZLEMENT'),
 ('200_Block_of_BUSH_ST', 'EMBEZZLEMENT')]

In [2]:
addDistRdd = (sc.textFile("/resources/data/MaprDataFiles/J_AddDist.csv")
                .map(lambda line : line.split(","))
                .map(lambda r: (r[1],r[0])))
addDistRdd.take(5)

[('100_Block_of_ROME_ST', 'INGLESIDE'),
 ('0_Block_of_SOUTHPARK_AV', 'SOUTHERN'),
 ('1900_Block_of_MISSION_ST', 'MISSION'),
 ('1400_Block_of_CLEMENT_ST', 'RICHMOND'),
 ('100_Block_of_BLUXOME_ST', 'SOUTHERN')]

In [3]:
addCatDistRdd = addCatRdd.join(addDistRdd)
addCatDistRdd.collect()

[('0_Block_of_SOUTHPARK_AV', ('BRIBERY', 'SOUTHERN')),
 ('1400_Block_of_CLEMENT_ST', ('BRIBERY', 'RICHMOND')),
 ('100_Block_of_BLUXOME_ST', ('BAD CHECKS', 'SOUTHERN')),
 ('1400_Block_of_VANDYKE_AV', ('BRIBERY', 'BAYVIEW')),
 ('1900_Block_of_MISSION_ST', ('BRIBERY', 'MISSION'))]

In [4]:
addCatDistRdd = addCatRdd.leftOuterJoin(addDistRdd)
addCatDistRdd.collect()

[('1600_Block_of_FILLMORE_ST', ('EMBEZZLEMENT', None)),
 ('SUTTER_ST/MASON_ST', ('EMBEZZLEMENT', None)),
 ('0_Block_of_SOUTHPARK_AV', ('BRIBERY', 'SOUTHERN')),
 ('1400_Block_of_CLEMENT_ST', ('BRIBERY', 'RICHMOND')),
 ('1500_Block_of_15TH_ST', ('EMBEZZLEMENT', None)),
 ('100_Block_of_BLUXOME_ST', ('BAD CHECKS', 'SOUTHERN')),
 ('800_Block_of_MARKET_ST', ('EMBEZZLEMENT', None)),
 ('200_Block_of_BUSH_ST', ('EMBEZZLEMENT', None)),
 ('1400_Block_of_VANDYKE_AV', ('BRIBERY', 'BAYVIEW')),
 ('100_Block_of_JEFFERSON_ST', ('EMBEZZLEMENT', None)),
 ('2000_Block_of_MARKET_ST', ('EMBEZZLEMENT', None)),
 ('1100_Block_of_SELBY_ST', ('EMBEZZLEMENT', None)),
 ('1900_Block_of_MISSION_ST', ('BRIBERY', 'MISSION'))]

In [5]:
addCatDistRdd = addCatRdd.rightOuterJoin(addDistRdd)
addCatDistRdd.collect()

[('300_Block_of_BERRY_ST', (None, 'SOUTHERN')),
 ('0_Block_of_SOUTHPARK_AV', ('BRIBERY', 'SOUTHERN')),
 ('1400_Block_of_CLEMENT_ST', ('BRIBERY', 'RICHMOND')),
 ('100_Block_of_BLUXOME_ST', ('BAD CHECKS', 'SOUTHERN')),
 ('1100_Block_of_MISSION_ST', (None, 'SOUTHERN')),
 ('1400_Block_of_VANDYKE_AV', ('BRIBERY', 'BAYVIEW')),
 ('0_Block_of_CHUMASERO_DR', (None, 'TARAVAL')),
 ('100_Block_of_ROME_ST', (None, 'INGLESIDE')),
 ('1900_Block_of_MISSION_ST', ('BRIBERY', 'MISSION'))]

In [6]:
addCatDistRdd = addCatRdd.fullOuterJoin(addDistRdd)
addCatDistRdd.collect()

[('1600_Block_of_FILLMORE_ST', ('EMBEZZLEMENT', None)),
 ('300_Block_of_BERRY_ST', (None, 'SOUTHERN')),
 ('SUTTER_ST/MASON_ST', ('EMBEZZLEMENT', None)),
 ('0_Block_of_SOUTHPARK_AV', ('BRIBERY', 'SOUTHERN')),
 ('1400_Block_of_CLEMENT_ST', ('BRIBERY', 'RICHMOND')),
 ('1500_Block_of_15TH_ST', ('EMBEZZLEMENT', None)),
 ('100_Block_of_BLUXOME_ST', ('BAD CHECKS', 'SOUTHERN')),
 ('800_Block_of_MARKET_ST', ('EMBEZZLEMENT', None)),
 ('1100_Block_of_MISSION_ST', (None, 'SOUTHERN')),
 ('200_Block_of_BUSH_ST', ('EMBEZZLEMENT', None)),
 ('1400_Block_of_VANDYKE_AV', ('BRIBERY', 'BAYVIEW')),
 ('100_Block_of_JEFFERSON_ST', ('EMBEZZLEMENT', None)),
 ('0_Block_of_CHUMASERO_DR', (None, 'TARAVAL')),
 ('2000_Block_of_MARKET_ST', ('EMBEZZLEMENT', None)),
 ('1100_Block_of_SELBY_ST', ('EMBEZZLEMENT', None)),
 ('100_Block_of_ROME_ST', (None, 'INGLESIDE')),
 ('1900_Block_of_MISSION_ST', ('BRIBERY', 'MISSION'))]

In [34]:
from pyspark.sql import Row
sfpdRDD = sc.textFile("./data/MaprDataFiles/sfpd.csv") \
        .map(lambda line:line.split(",")).map(lambda x: ((x[4],x[5]), (x[1], x[2],x[6], x[7],x[8])))
sfpdRDD.take(3)

[(('7/9/15', '23:45'),
  ('OTHER_OFFENSES',
   'POSSESSION_OF_BURGLARY_TOOLS',
   'CENTRAL',
   'ARREST/BOOKED',
   'JACKSON_ST/POWELL_ST')),
 (('7/9/15', '23:45'),
  ('LARCENY/THEFT',
   'PETTY_THEFT_OF_PROPERTY',
   'CENTRAL',
   'NONE',
   '300_Block_of_POWELL_ST')),
 (('7/9/15', '23:45'),
  ('OTHER_OFFENSES',
   'DRIVERS_LICENSE/SUSPENDED_OR_REVOKED',
   'CENTRAL',
   'ARREST/BOOKED',
   'JACKSON_ST/POWELL_ST'))]

In [35]:
sfpdRDD.join(sfpdRDD).take(5)

[(('3/5/14', '11:39'),
  (('LARCENY/THEFT',
    'GRAND_THEFT_FROM_LOCKED_AUTO',
    'CENTRAL',
    'NONE',
    '400_Block_of_STOCKTON_ST'),
   ('LARCENY/THEFT',
    'GRAND_THEFT_FROM_LOCKED_AUTO',
    'CENTRAL',
    'NONE',
    '400_Block_of_STOCKTON_ST'))),
 (('1/22/15', '13:06'),
  (('SUSPICIOUS_OCC',
    'INVESTIGATIVE_DETENTION',
    'NORTHERN',
    'UNFOUNDED',
    'SANCHEZ_ST/14TH_ST'),
   ('SUSPICIOUS_OCC',
    'INVESTIGATIVE_DETENTION',
    'NORTHERN',
    'UNFOUNDED',
    'SANCHEZ_ST/14TH_ST'))),
 (('7/16/13', '11:50'),
  (('BURGLARY',
    'BURGLARY/STORE_UNDER_CONSTRUCTION/UNLAWFUL_ENTRY',
    'SOUTHERN',
    'NONE',
    '0_Block_of_ZOE_ST'),
   ('BURGLARY',
    'BURGLARY/STORE_UNDER_CONSTRUCTION/UNLAWFUL_ENTRY',
    'SOUTHERN',
    'NONE',
    '0_Block_of_ZOE_ST'))),
 (('4/7/14', '13:25'),
  (('NON-CRIMINAL',
    'AIDED_CASE/MENTAL_DISTURBED',
    'MISSION',
    'PSYCHOPATHIC_CASE',
    '1100_Block_of_POTRERO_AV'),
   ('NON-CRIMINAL',
    'AIDED_CASE/MENTAL_DISTURBED',
    '

In [48]:
rdd1 =  sc.parallelize((("foo", 1), ("bar", 2), ("baz", 3)))
rdd2 =  sc.parallelize((("foo", 4), ("bar", 5), ("bar", 6)))

In [49]:
# inner join
rdd1.join(rdd2).collect()

[('bar', (2, 5)), ('bar', (2, 6)), ('foo', (1, 4))]

In [39]:
# left outer
rdd1.leftOuterJoin(rdd2).collect()

[('bar', (2, 5)), ('bar', (2, 6)), ('baz', (3, None)), ('foo', (1, 4))]

In [50]:
# right outer
rdd1.rightOuterJoin(rdd2).collect()

[('bar', (2, 5)), ('bar', (2, 6)), ('foo', (1, 4))]

In [51]:
# full outer join
rdd1.fullOuterJoin(rdd2).collect()

[('bar', (2, 5)), ('bar', (2, 6)), ('baz', (3, None)), ('foo', (1, 4))]

In [52]:
# cartesian
rdd1.cartesian(rdd2).collect()

[(('foo', 1), ('foo', 4)),
 (('foo', 1), ('bar', 5)),
 (('foo', 1), ('bar', 6)),
 (('bar', 2), ('foo', 4)),
 (('bar', 2), ('bar', 5)),
 (('bar', 2), ('bar', 6)),
 (('baz', 3), ('foo', 4)),
 (('baz', 3), ('bar', 5)),
 (('baz', 3), ('bar', 6))]

In [69]:
# cogroup
rdd3 = rdd1.cogroup(rdd2)

In [81]:
[(k[0],tuple(k[1][0]),tuple(k[1][1])) for k in rdd3.collect()]

[('bar', (2,), (5, 6)), ('baz', (3,), ()), ('foo', (1,), (4,))]

In [92]:
sfpdRDD.countByKey()

defaultdict(int,
            {('11/28/13', '10:30'): 1,
             ('6/12/14', '17:30'): 8,
             ('11/25/13', '20:30'): 1,
             ('1/8/13', '4:00'): 5,
             ('6/7/13', '15:45'): 2,
             ('1/8/14', '18:01'): 1,
             ('2/19/15', '16:55'): 1,
             ('3/27/15', '13:08'): 1,
             ('3/3/15', '18:40'): 2,
             ('2/15/13', '9:24'): 1,
             ('1/27/15', '8:50'): 2,
             ('6/11/13', '11:10'): 3,
             ('7/10/13', '2:30'): 1,
             ('1/18/15', '19:30'): 4,
             ('6/28/15', '22:41'): 2,
             ('6/26/15', '8:30'): 4,
             ('12/7/13', '19:47'): 2,
             ('5/23/14', '12:59'): 1,
             ('9/29/13', '16:22'): 1,
             ('10/13/14', '0:01'): 5,
             ('1/9/15', '10:35'): 2,
             ('7/12/14', '23:45'): 2,
             ('9/25/14', '12:38'): 1,
             ('12/1/14', '9:05'): 2,
             ('2/8/13', '17:05'): 1,
             ('3/11/15', '18:14'): 1,
     

In [93]:
sfpdRDD.countByValue()

defaultdict(int,
            {(('5/8/14', '9:55'),
              ('STOLEN_PROPERTY',
               'STOLEN_PROPERTY/POSSESSION_WITH_KNOWLEDGE/RECEIVING',
               'BAYVIEW',
               'ARREST/BOOKED',
               '200_Block_of_GAVEN_ST')): 1,
             (('3/22/15', '20:20'),
              ('LARCENY/THEFT',
               'PETTY_THEFT_FROM_UNLOCKED_AUTO',
               'TARAVAL',
               'NONE',
               '1100_Block_of_JUNIPERO_SERRA_BL')): 1,
             (('7/7/13', '21:30'),
              ('VANDALISM',
               'MALICIOUS_MISCHIEF/VANDALISM',
               'CENTRAL',
               'NONE',
               '400_Block_of_GEARY_ST')): 1,
             (('2/23/13', '11:00'),
              ('LARCENY/THEFT',
               'GRAND_THEFT_FROM_LOCKED_AUTO',
               'NORTHERN',
               'NONE',
               '0_Block_of_LAGUNA_ST')): 1,
             (('8/31/13', '14:00'),
              ('LARCENY/THEFT',
               'GRAND_THEFT_FROM_A_BUILD

In [94]:
rdd1.countByKey()

defaultdict(int, {'bar': 1, 'baz': 1, 'foo': 1})

In [95]:
rdd1.countByValue()

defaultdict(int, {('bar', 2): 1, ('baz', 3): 1, ('foo', 1): 1})

In [96]:
rdd1.collect()

[('foo', 1), ('bar', 2), ('baz', 3)]

In [107]:
rdd4 = sc.parallelize([('foo', 1), ('bar', 2), ('baz', 3), ('baz', 4), ('baz', 2)])

In [108]:
rdd4.collect()

[('foo', 1), ('bar', 2), ('baz', 3), ('baz', 4), ('baz', 2)]

In [100]:
rdd4.countByKey()

defaultdict(int, {'bar': 1, 'baz': 3, 'foo': 1})

In [101]:
rdd4.countByValue()

defaultdict(int, {('bar', 2): 1, ('baz', 3): 2, ('baz', 4): 1, ('foo', 1): 1})

In [109]:
rdd4.collectAsMap()

{'bar': 2, 'baz': 2, 'foo': 1}

In [110]:
rdd4.lookup("bar")

[2]

In [111]:
rdd4.lookup("baz")

[3, 4, 2]

In [112]:
rdd4.lookup("foo")

[1]

In [113]:
rdd4.countByKey()

defaultdict(int, {'bar': 1, 'baz': 3, 'foo': 1})

In [120]:
rdd4.groupByKey().map(lambda x: (x[0],len(x[1]))).collect()

[('foo', 1), ('bar', 1), ('baz', 3)]

In [121]:
rdd4.groupByKey().map(lambda x: (x[0],list(x[1]))).collect()

[('foo', [1]), ('bar', [2]), ('baz', [3, 4, 2])]

In [118]:
len(a)

3

### Coalesce vs Repartition
The repartition algorithm does a full shuffle and creates new partitions with data that's distributed evenly. Let's create an RDD with the numbers from 1 to 12.

a = sc.range(1,13,1,4)

RDD contains 4 partitions on my machine.

a.getNumPartitions() // => 4

Here is how the data is divided on the partitions:

a.glom().collect()

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]

Let's do a full-shuffle with the repartition method and get this data on two nodes.

> *a.repartition(2).glom().collect()*

> *[[1, 4, 7, 10], [2, 3, 5, 6, 8, 9, 11, 12]]*

The repartition method makes new partitions and evenly distributes the data in the new partitions (the data distribution is more even for larger data sets).

> *a.coalesce(2).glom().collect()*

> *[[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12]]*

**Difference between coalesce and repartition?**

Coalesce uses existing partitions to minimize the amount of data that's shuffled. 
Repartition creates new partitions and does a full shuffle. 

Coalesce results in partitions with different amounts of data (sometimes partitions that have much different sizes) and Repartition results in roughly equal sized partitions.

**Is coalesce or repartition faster?**

coalesce may run faster than repartition, but unequal sized partitions are generally slower to work with than equal sized partitions. You'll usually need to repartition datasets after filtering a large data set. I've found repartition to be faster overall because Spark is built to work with equal sized partitions.

Read this blog post if you'd like even more details.

In [142]:
a = sc.range(1,13,1,4)

In [143]:
a.getNumPartitions()

4

In [144]:
a.glom().collect()

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]

In [145]:
a.coalesce(2).glom().collect()

[[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12]]

In [147]:
a.repartition(2).glom().collect()

[[1, 4, 7, 10], [2, 3, 5, 6, 8, 9, 11, 12]]

In [159]:
sfpdRDD = (sc.textFile("/resources/data/MaprDataFiles/sfpd.csv", 16)
           .map(lambda x:x.split(","))
           .map(lambda x: (x[6],1)))


In [160]:
sfpdRDD.getNumPartitions()

16

In [164]:
sfpdRDD.reduceByKey(lambda x,y: x+y).toDebugString()

b'(16) PythonRDD[462] at RDD at PythonRDD.scala:43 []\n |   MapPartitionsRDD[461] at mapPartitions at PythonRDD.scala:374 []\n |   ShuffledRDD[460] at partitionBy at null:-1 []\n +-(16) PairwiseRDD[459] at reduceByKey at <ipython-input-164-323e2486cdb6>:1 []\n    |   PythonRDD[458] at reduceByKey at <ipython-input-164-323e2486cdb6>:1 []\n    |   MapPartitionsRDD[452] at textFile at null:-1 []\n    |   /resources/data/MaprDataFiles/sfpd.csv HadoopRDD[451] at textFile at null:-1 []'

In [162]:
a.getNumPartitions()

16

In [172]:
sfpdRDD.getNumPartitions()

16

In [190]:
catAdd = sc.textFile("/resources/data/MaprDataFiles/J_AddCat.csv").map(lambda x: x.split(",")).map(lambda x: (x(1),x(0)))
distAdd = sc.textFile("/resources/data/MaprDataFiles/J_AddDist.csv").map(lambda x: x.split(",")).map(lambda x: (x(1),x(0)))

In [191]:
catAdd.getNumPartitions()

2

In [206]:
pairs = sc.parallelize([1, 2, 3, 4, 2, 4, 1]).map(lambda x: (x, x))
pairs.collect()

[(1, 1), (2, 2), (3, 3), (4, 4), (2, 2), (4, 4), (1, 1)]

In [207]:
pairs.getNumPartitions()

16

In [208]:
pairs.glom().collect()

[[],
 [],
 [(1, 1)],
 [],
 [(2, 2)],
 [],
 [(3, 3)],
 [],
 [],
 [(4, 4)],
 [],
 [(2, 2)],
 [],
 [(4, 4)],
 [],
 [(1, 1)]]

In [223]:
sets = pairs.partitionBy(4, hash).glom()

In [224]:
sets.collect()

[[(4, 4), (4, 4)], [(1, 1), (1, 1)], [(2, 2), (2, 2)], [(3, 3)]]

In [183]:
distAdd.getNumPartitions()

2

In [176]:
catJdist=catAdd.join(distAdd,8)

In [177]:
catJdist.getNumPartitions()

8

In [179]:
catJdist.toDebugString()

b'(8) PythonRDD[474] at RDD at PythonRDD.scala:43 []\n |  MapPartitionsRDD[473] at mapPartitions at PythonRDD.scala:374 []\n |  ShuffledRDD[472] at partitionBy at null:-1 []\n +-(4) PairwiseRDD[471] at join at <ipython-input-176-2281bd05c978>:1 []\n    |  PythonRDD[470] at join at <ipython-input-176-2281bd05c978>:1 []\n    |  UnionRDD[469] at union at null:-1 []\n    |  PythonRDD[467] at RDD at PythonRDD.scala:43 []\n    |  MapPartitionsRDD[464] at textFile at null:-1 []\n    |  /resources/data/MaprDataFiles/J_AddCat.csv HadoopRDD[463] at textFile at null:-1 []\n    |  PythonRDD[468] at RDD at PythonRDD.scala:43 []\n    |  MapPartitionsRDD[466] at textFile at null:-1 []\n    |  /resources/data/MaprDataFiles/J_AddDist.csv HadoopRDD[465] at textFile at null:-1 []'

# Working with DataFrames

In [227]:
from pyspark.sql import Row

sfpdRDD = (sc.textFile("/resources/data/MaprDataFiles/sfpd.csv")
            .map(lambda x: x.split(","))
            .map(lambda i: Row(IncidentNum = int(i[0]), Category = i[1], Descript = i[2], DayOfWeek = i[3]
                         , Date = i[4], Time = i[5], PdDistrict = i[6], Resolution = i[7] 
                         , Address = i[8], XCoord = i[9], YCoord = i[10], PdID = int(i[11]))))
sfpdRDD.take(5)

[Row(Address='JACKSON_ST/POWELL_ST', Category='OTHER_OFFENSES', Date='7/9/15', DayOfWeek='Thursday', Descript='POSSESSION_OF_BURGLARY_TOOLS', IncidentNum=150599321, PdDistrict='CENTRAL', PdID=15059900000000, Resolution='ARREST/BOOKED', Time='23:45', XCoord='-122.4099006', YCoord='37.79561712'),
 Row(Address='300_Block_of_POWELL_ST', Category='LARCENY/THEFT', Date='7/9/15', DayOfWeek='Thursday', Descript='PETTY_THEFT_OF_PROPERTY', IncidentNum=156168837, PdDistrict='CENTRAL', PdID=15616900000000, Resolution='NONE', Time='23:45', XCoord='-122.4083843', YCoord='37.78782711'),
 Row(Address='JACKSON_ST/POWELL_ST', Category='OTHER_OFFENSES', Date='7/9/15', DayOfWeek='Thursday', Descript='DRIVERS_LICENSE/SUSPENDED_OR_REVOKED', IncidentNum=150599321, PdDistrict='CENTRAL', PdID=15059900000000, Resolution='ARREST/BOOKED', Time='23:45', XCoord='-122.4099006', YCoord='37.79561712'),
 Row(Address='MASONIC_AV/GOLDEN_GATE_AV', Category='OTHER_OFFENSES', Date='7/9/15', DayOfWeek='Thursday', Descript='D

In [228]:
sfpdRDD.top(5)

[Row(Address='ZOE_ST/WELSH_ST', Category='SEX_OFFENSES/FORCIBLE', Date='9/29/13', DayOfWeek='Sunday', Descript='PENETRATION/FORCED/WITH_OBJECT', IncidentNum=130832301, PdDistrict='SOUTHERN', PdID=13083200000000, Resolution='NONE', Time='14:00', XCoord='-122.3963085', YCoord='37.78009677'),
 Row(Address='ZOE_ST/WELSH_ST', Category='ROBBERY', Date='9/29/13', DayOfWeek='Sunday', Descript='ROBBERY/ARMED_WITH_A_GUN', IncidentNum=130832301, PdDistrict='SOUTHERN', PdID=13083200000000, Resolution='NONE', Time='14:00', XCoord='-122.3963085', YCoord='37.78009677'),
 Row(Address='ZOE_ST/WELSH_ST', Category='LARCENY/THEFT', Date='5/13/15', DayOfWeek='Wednesday', Descript='GRAND_THEFT_FROM_UNLOCKED_AUTO', IncidentNum=156162215, PdDistrict='SOUTHERN', PdID=15616200000000, Resolution='NONE', Time='8:00', XCoord='-122.3963085', YCoord='37.78009677'),
 Row(Address='ZOE_ST/WELSH_ST', Category='LARCENY/THEFT', Date='12/29/13', DayOfWeek='Sunday', Descript='GRAND_THEFT_FROM_LOCKED_AUTO', IncidentNum=14600

In [ ]:
sfpdDF = sfpdRDD.toDF()

In [237]:
sfpdDF.registerTempTable("sfpd")

In [241]:
incByDist = sfpdDF.groupby(sfpdDF['PdDistrict']).count().orderBy("`count`", ascending = False)
incByDist.show(5)

+----------+-----+
|PdDistrict|count|
+----------+-----+
|  SOUTHERN|73308|
|   MISSION|50164|
|  NORTHERN|46877|
|   CENTRAL|41914|
|   BAYVIEW|36111|
+----------+-----+
only showing top 5 rows



In [244]:
from pyspark.sql.functions import col
sfpdDF.groupby(sfpdDF['PdDistrict']).count().sort(col("count").desc()).show(5)

+----------+-----+
|PdDistrict|count|
+----------+-----+
|  SOUTHERN|73308|
|   MISSION|50164|
|  NORTHERN|46877|
|   CENTRAL|41914|
|   BAYVIEW|36111|
+----------+-----+
only showing top 5 rows



In [245]:
sqlContext.sql("""
                SELECT PdDistrict, Count(1) as IncCount
                FROM sfpd
                GROUP BY PdDistrict
                ORDER BY IncCount DESC
                LIMIT 5
               """).show()

+----------+--------+
|PdDistrict|IncCount|
+----------+--------+
|  SOUTHERN|   73308|
|   MISSION|   50164|
|  NORTHERN|   46877|
|   CENTRAL|   41914|
|   BAYVIEW|   36111|
+----------+--------+



In [256]:
# top10Res
res = sfpdDF.groupby(sfpdDF['Resolution']).count().sort(col("count").desc())

In [265]:
res.write.format("json").mode("append").save("/resources/data/MaprDataFiles/Outputs/sfpdtopresjson")

In [266]:
%%bash
ls /resources/data/MaprDataFiles/Outputs/sfpdtopresjson/

part-r-00000-29a06e3b-1916-4f66-b378-b64bc6358d26
part-r-00000-5873c28a-cc37-4ad3-8187-75fbe84b1a42
part-r-00001-29a06e3b-1916-4f66-b378-b64bc6358d26
part-r-00001-5873c28a-cc37-4ad3-8187-75fbe84b1a42
part-r-00002-29a06e3b-1916-4f66-b378-b64bc6358d26
part-r-00002-5873c28a-cc37-4ad3-8187-75fbe84b1a42
part-r-00003-29a06e3b-1916-4f66-b378-b64bc6358d26
part-r-00003-5873c28a-cc37-4ad3-8187-75fbe84b1a42
part-r-00004-29a06e3b-1916-4f66-b378-b64bc6358d26
part-r-00004-5873c28a-cc37-4ad3-8187-75fbe84b1a42
part-r-00005-29a06e3b-1916-4f66-b378-b64bc6358d26
part-r-00005-5873c28a-cc37-4ad3-8187-75fbe84b1a42
part-r-00006-29a06e3b-1916-4f66-b378-b64bc6358d26
part-r-00006-5873c28a-cc37-4ad3-8187-75fbe84b1a42
part-r-00007-29a06e3b-1916-4f66-b378-b64bc6358d26
part-r-00007-5873c28a-cc37-4ad3-8187-75fbe84b1a42
part-r-00008-29a06e3b-1916-4f66-b378-b64bc6358d26
part-r-00008-5873c28a-cc37-4ad3-8187-75fbe84b1a42
part-r-00009-29a06e3b-1916-4f66-b378-b64bc6358d26
part-r-00009-5873c28a-cc37-4ad3-8187-75fbe84b1a42


In [267]:
aadf = sqlContext.read.json("/resources/data/MaprDataFiles/Outputs/sfpdtopresjson")

In [271]:
aadf.select("Resolution").distinct().count()

17

In [268]:
%%bash
cat /resources/data/MaprDataFiles/Outputs/sfpdtopresjson/part-r-00003-29a06e3b-1916-4f66-b378-b64bc6358d26

{"Resolution":"PSYCHOPATHIC_CASE","count":8344}


In [272]:
aadf.select("Resolution").count()

34

In [274]:
#  'overwrite', 'append', 'ignore', 'error'
res.write.mode("append").save("/resources/data/MaprDataFiles/Outputs/sfpdresdefault")

In [278]:
aaa = sqlContext.read.parquet("/resources/data/MaprDataFiles/Outputs/sfpdresdefault")

In [279]:
aaa.show()

+--------------------+------+
|          Resolution| count|
+--------------------+------+
|      JUVENILE_CITED|  1039|
|        ARREST/CITED| 22925|
|           UNFOUNDED|  4551|
|      NOT_PROSECUTED|   280|
|             LOCATED|  6878|
|COMPLAINANT_REFUS...|  4215|
|DISTRICT_ATTORNEY...|   473|
| JUVENILE_ADMONISHED|   344|
|                NONE|243538|
|       ARREST/BOOKED| 86766|
|   JUVENILE_DIVERTED|   120|
|     JUVENILE_BOOKED|  2381|
|PROSECUTED_BY_OUT...|   630|
|   PSYCHOPATHIC_CASE|  8344|
|EXCEPTIONAL_CLEAR...|  1134|
|CLEARED-CONTACT_J...|   152|
|PROSECUTED_FOR_LE...|     5|
+--------------------+------+



In [282]:
sfpdDF.select("Date").show(5)

+------+
|  Date|
+------+
|7/9/15|
|7/9/15|
|7/9/15|
|7/9/15|
|7/9/15|
+------+
only showing top 5 rows



In [289]:
def getYear(datecol):
    a = datecol.split("/")
    return "20"+a[2]



2015


In [286]:
a = '7/9/15'
date = a.split("/")
date[2]

'15'

In [290]:
sqlContext.udf.register("year", getYear)

In [291]:
sqlContext.sql("select year(date) as year from sfpd").show()

+----+
|year|
+----+
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
|2015|
+----+
only showing top 20 rows



In [292]:
sqlContext.udf.register("yyyy", lambda x: x.split("/"))

In [293]:
sqlContext.sql("select yyyy(date) as year from sfpd").show()

+----------+
|      year|
+----------+
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
|[7, 9, 15]|
+----------+
only showing top 20 rows



In [299]:
sfpdDF.filter("year(date) = 2014").show(10)

+--------------------+--------------+--------+---------+--------------------+-----------+----------+--------------+----------+-----+------------+-----------+
|             Address|      Category|    Date|DayOfWeek|            Descript|IncidentNum|PdDistrict|          PdID|Resolution| Time|      XCoord|     YCoord|
+--------------------+--------------+--------+---------+--------------------+-----------+----------+--------------+----------+-----+------------+-----------+
|0_Block_of_UNITED...|       ASSAULT|12/31/14|Wednesday|             BATTERY|  150000011|  SOUTHERN|15000000000000|      NONE|23:55|-122.4143179|37.77994441|
|400_Block_of_POWE...|  NON-CRIMINAL|12/31/14|Wednesday|       LOST_PROPERTY|  156006027|   CENTRAL|15600600000000|      NONE|23:50|-122.4084319|37.78877727|
|700_Block_of_MONT...|       ASSAULT|12/31/14|Wednesday|             BATTERY|  150000130|   CENTRAL|15000000000000|      NONE|23:50|-122.4033402|37.79599189|
|  FOLSOM_ST/SPEAR_ST|     VANDALISM|12/31/14|Wednes

In [300]:
sfpdRDD.toDebugString()

b'(2) PythonRDD[798] at RDD at PythonRDD.scala:43 []\n |  MapPartitionsRDD[565] at textFile at null:-1 []\n |  /resources/data/MaprDataFiles/sfpd.csv HadoopRDD[564] at textFile at null:-1 []'

In [301]:
sfpdRDD.toDebugString()

b'(2) PythonRDD[798] at RDD at PythonRDD.scala:43 []\n |  MapPartitionsRDD[565] at textFile at null:-1 []\n |  /resources/data/MaprDataFiles/sfpd.csv HadoopRDD[564] at textFile at null:-1 []'